# TrustWeight Async Federated Learning — Google Colab Version

This notebook is organized with one cell per original source file,
and is optimized for running on **Google Colab** with all results
stored on **Google Drive**.

**How to use on Colab:**

1. Upload this notebook to Colab.
2. Run the first cell to mount Google Drive and set your base path.
3. All subsequent paths (data, logs, models, results) will be created
   relative to that base directory.


In [51]:
! pip install pytorch_lightning

In [52]:

# @title Mount Google Drive and set base project directory
from google.colab import drive
import os
from pathlib import Path

# Mount your Google Drive
drive.mount('/content/drive')

# Base directory INSIDE your Drive where everything will be stored.
# You can change this to any folder you like.
BASE_DIR = "/content/drive/MyDrive/TrustWeightColab"  # @param {type:"string"}

BASE_PATH = Path(BASE_DIR).expanduser()
BASE_PATH.mkdir(parents=True, exist_ok=True)
os.chdir(BASE_PATH)
os.environ["TRUSTWEIGHT_BASE_DIR"] = str(BASE_PATH)
print("Working directory set to:", os.getcwd())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Working directory set to: /content/drive/MyDrive/TrustWeightColab


## config.yaml

Below is the YAML configuration used by the code.


In [53]:

from pathlib import Path
import textwrap

config_path = Path(BASE_DIR) / "config.yaml"
config_text = f"""
data:
  dataset: "cifar10"
  data_dir: "{(Path(BASE_DIR) / 'data').as_posix()}"
  num_classes: 10

clients:
  total: 20
  concurrent: 10
  local_epochs: 1
  batch_size: 128
  lr: 0.005
  weight_decay: 0.001
  grad_clip: 5.0

  # Straggler simulation
  struggle_percent: 0         # % of clients treated as slow
  delay_slow_range: [0.8, 2.0]  # seconds
  delay_fast_range: [0.0, 0.2]
  jitter_per_round: 0.05
  fix_delays_per_client: true   # keep each client's slow/fast identity

eval:
  interval_seconds: 15          # (not used for logging now but kept for compatibility)
  target_accuracy: 0.80         # stop training when achieved

train:
  max_rounds: 500              # stop after this many aggregations
  update_clip_norm: 5.0          # clip oversized update vectors before aggregation

partition_alpha: 1000            # Dirichlet α for non-IID partitioning
seed: 42

server_runtime:
  client_delay: 0.0             # fixed delay added to each client

io:
  checkpoints_dir: "{(Path(BASE_DIR) / 'checkpoints' / 'TrustWeight').as_posix()}"
  logs: "{(Path(BASE_DIR) / 'logs').as_posix()}"
  results: "{(Path(BASE_DIR) / 'results').as_posix()}"
  # Global server-level training log (one row per aggregation)
  global_log_csv: "{(Path(BASE_DIR) / 'logs' / 'TrustWeight.csv').as_posix()}"
  # Per-client participation log (one row per client per aggregation)
  client_participation_csv: "{(Path(BASE_DIR) / 'logs' / 'TrustWeightClientParticipation.csv').as_posix()}"
  final_model_path: "{(Path(BASE_DIR) / 'results' / 'TrustWeightModel.pt').as_posix()}"
"""

config_path.write_text(textwrap.dedent(config_text).lstrip())
print(f"config.yaml written to {config_path}")


config.yaml written to /content/drive/MyDrive/TrustWeightColab/config.yaml


In [54]:

# ===== config.py =====

from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, Tuple
import os

import yaml


# ----------------------------- base paths ------------------------------------
BASE_PATH = Path(
    os.environ.get("TRUSTWEIGHT_BASE_DIR", globals().get("BASE_DIR", "."))
).expanduser()
BASE_PATH.mkdir(parents=True, exist_ok=True)


def _resolve_to_base(path_like: str | Path) -> str:
    """Return absolute path under BASE_PATH unless already absolute."""
    p = Path(path_like)
    if p.is_absolute():
        return str(p)
    return str(BASE_PATH / p)


# ----------------------------- dataclasses -----------------------------------


@dataclass
class DataConfig:
    dataset: str
    data_dir: str
    num_classes: int


@dataclass
class ClientsConfig:
    total: int
    concurrent: int
    local_epochs: int
    batch_size: int
    lr: float
    weight_decay: float
    grad_clip: float
    struggle_percent: int
    delay_slow_range: Tuple[float, float]
    delay_fast_range: Tuple[float, float]
    jitter_per_round: float
    fix_delays_per_client: bool


@dataclass
class EvalConfig:
    interval_seconds: int
    target_accuracy: float


@dataclass
class TrainConfig:
    max_rounds: int
    update_clip_norm: float


@dataclass
class ServerRuntimeConfig:
    client_delay: float


@dataclass
class IOConfig:
    checkpoints_dir: str
    logs_dir: str
    results_dir: str
    global_log_csv: str
    client_participation_csv: str
    final_model_path: str


@dataclass
class GlobalConfig:
    data: DataConfig
    clients: ClientsConfig
    eval: EvalConfig
    train: TrainConfig
    partition_alpha: float
    seed: int
    server_runtime: ServerRuntimeConfig
    io: IOConfig


# ----------------------------- loader ----------------------------------------


def _as_tuple(x: Any) -> Tuple[float, float]:
    if isinstance(x, (list, tuple)) and len(x) == 2:
        return float(x[0]), float(x[1])
    raise ValueError(f"Expected length-2 sequence, got {x!r}")


def load_config(path: str | Path | None = None) -> GlobalConfig:
    cfg_path = Path(path) if path is not None else BASE_PATH / "config.yaml"
    with cfg_path.open("r") as f:
        raw: Dict[str, Any] = yaml.safe_load(f)

    data_cfg = DataConfig(
        dataset=raw["data"]["dataset"],
        data_dir=_resolve_to_base(raw["data"]["data_dir"]),
        num_classes=int(raw["data"]["num_classes"]),
    )

    clients_section = raw["clients"]
    clients_cfg = ClientsConfig(
        total=int(clients_section["total"]),
        concurrent=int(clients_section["concurrent"]),
        local_epochs=int(clients_section["local_epochs"]),
        batch_size=int(clients_section["batch_size"]),
        lr=float(clients_section["lr"]),
        weight_decay=float(clients_section.get("weight_decay", 5e-4)),
        grad_clip=float(clients_section.get("grad_clip", 5.0)),
        struggle_percent=int(clients_section.get("struggle_percent", 0)),
        delay_slow_range=_as_tuple(clients_section.get("delay_slow_range", [0.8, 2.0])),
        delay_fast_range=_as_tuple(clients_section.get("delay_fast_range", [0.0, 0.2])),
        jitter_per_round=float(clients_section.get("jitter_per_round", 0.0)),
        fix_delays_per_client=bool(clients_section.get("fix_delays_per_client", True)),
    )

    eval_cfg = EvalConfig(
        interval_seconds=int(raw["eval"]["interval_seconds"]),
        target_accuracy=float(raw["eval"]["target_accuracy"]),
    )

    train_cfg = TrainConfig(
        max_rounds=int(raw["train"]["max_rounds"]),
        update_clip_norm=float(raw["train"].get("update_clip_norm", 10.0)),
    )

    server_runtime_cfg = ServerRuntimeConfig(
        client_delay=float(raw["server_runtime"].get("client_delay", 0.0))
    )

    io_section = raw["io"]
    io_cfg = IOConfig(
        checkpoints_dir=_resolve_to_base(io_section["checkpoints_dir"]),
        logs_dir=_resolve_to_base(io_section["logs"]),
        results_dir=_resolve_to_base(io_section["results"]),
        global_log_csv=_resolve_to_base(io_section["global_log_csv"]),
        client_participation_csv=_resolve_to_base(io_section["client_participation_csv"]),
        final_model_path=_resolve_to_base(io_section["final_model_path"]),
    )

    cfg = GlobalConfig(
        data=data_cfg,
        clients=clients_cfg,
        eval=eval_cfg,
        train=train_cfg,
        partition_alpha=float(raw.get("partition_alpha", 0.5)),
        seed=int(raw.get("seed", 42)),
        server_runtime=server_runtime_cfg,
        io=io_cfg,
    )
    return cfg


In [55]:
# ===== helper.py =====

from __future__ import annotations

import random
import numpy as np
import torch


def set_seed(seed: int = 42) -> None:
    """Seed all RNGs used in this project.

    Setting a global seed helps to ensure reproducible results.  This
    function touches Python's built‑in random module, NumPy, and
    PyTorch's CPU and GPU RNGs.  Deterministic behaviour in cuDNN
    kernels is also enabled.

    Parameters
    ----------
    seed:
        The random seed to use.  Defaults to ``42``.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    # When running on GPUs you may have more than one device
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # Deterministic behaviour comes at a performance cost but
    # reproducibility is more important for experimentation.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def get_device() -> torch.device:
    """Return the first available computation device.

    Tries to use CUDA if available, otherwise falls back to MPS
    (Apple Silicon) and finally the CPU.

    Returns
    -------
    device:
        A PyTorch ``torch.device`` object indicating where tensors
        should be allocated.
    """
    if torch.cuda.is_available():
        return torch.device("cuda")
    # MPS stands for Metal Performance Shaders.  It is the backend
    # available on Apple Silicon systems for GPU acceleration.
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")


In [56]:
# ===== model.py =====

# Utilities for building models and converting parameters
from typing import Dict, List
import torch
import torch.nn as nn
from torchvision import models


def build_resnet18(num_classes: int = 10, pretrained: bool = False) -> nn.Module:
    """Create a ResNet-18 tailored for CIFAR-size inputs."""
    m = models.resnet18(weights=None)
    m.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    m.maxpool = nn.Identity()

    in_features = m.fc.in_features
    m.fc = nn.Linear(in_features, num_classes)
    m.num_classes = num_classes
    return m


def state_to_list(state: Dict[str, torch.Tensor]) -> List[torch.Tensor]:
    """Flatten a state_dict to a list of tensors on CPU."""
    return [t.detach().cpu().clone() for _, t in state.items()]


def list_to_state(template: Dict[str, torch.Tensor], arrs: List[torch.Tensor]) -> Dict[str, torch.Tensor]:
    """Rebuild a state_dict from a list of tensors using a template for keys/dtypes/devices."""
    out: Dict[str, torch.Tensor] = {}
    for (k, v), a in zip(template.items(), arrs):
        out[k] = a.to(v.device).type_as(v)
    return out


In [57]:

# ===== partitioning.py =====

import os
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import Subset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Any


BASE_PATH = Path(os.environ.get("TRUSTWEIGHT_BASE_DIR", ".")).expanduser()
DEFAULT_DATA_DIR = str(BASE_PATH / "data")
DEFAULT_RESULTS_DIR = BASE_PATH / "results"


class DataDistributor:
    def __init__(self, dataset_name: str, data_dir: str = DEFAULT_DATA_DIR):
        """
        Flexible data distributor for federated learning experiments.

        Args:
            dataset_name (str): Name of dataset ('CIFAR10', 'MNIST', etc.)
            data_dir (str): Directory to store data.
        """
        self.dataset_name = dataset_name.lower()
        data_root = Path(data_dir)
        if not data_root.is_absolute():
            data_root = BASE_PATH / data_root
        self.data_dir = str(data_root)
        self.train_dataset, self.test_dataset, self.num_classes = self._load_dataset()
        self.partitions = None

    def _load_dataset(self) -> Tuple[Any, Any, int]:
        """Load supported torchvision datasets."""
        transform = transforms.Compose([transforms.ToTensor()])

        if self.dataset_name == "cifar10":
            train = datasets.CIFAR10(self.data_dir, train=True, download=True, transform=transform)
            test = datasets.CIFAR10(self.data_dir, train=False, download=True, transform=transform)
            num_classes = 10

        elif self.dataset_name == "mnist":
            train = datasets.MNIST(self.data_dir, train=True, download=True, transform=transform)
            test = datasets.MNIST(self.data_dir, train=False, download=True, transform=transform)
            num_classes = 10

        elif self.dataset_name == "fashionmnist":
            train = datasets.FashionMNIST(self.data_dir, train=True, download=True, transform=transform)
            test = datasets.FashionMNIST(self.data_dir, train=False, download=True, transform=transform)
            num_classes = 10

        else:
            raise ValueError(f"Dataset '{self.dataset_name}' is not supported yet.")

        return train, test, num_classes

    def distribute_data(self, num_clients: int, alpha: float = 0.5, seed: int = 42) -> Dict[int, List[int]]:
        """
        Perform Dirichlet-based data partitioning across clients (Non-IID).

        Args:
            num_clients (int): Number of clients.
            alpha (float): Dirichlet distribution parameter (smaller = more non-IID).
            seed (int): Random seed for reproducibility.
        """
        np.random.seed(seed)
        targets = np.array(self.train_dataset.targets)
        self.partitions = {i: [] for i in range(num_clients)}

        for cls in range(self.num_classes):
            idxs = np.where(targets == cls)[0]
            # Shuffle indices for this class
            np.random.shuffle(idxs)
            # Sample proportions from a Dirichlet distribution
            proportions = np.random.dirichlet(alpha=np.repeat(alpha, num_clients))
            # Convert proportions to integer counts (floor) for each client
            int_props = np.floor(proportions * len(idxs)).astype(int)
            # Assign counts to clients
            start = 0
            for client_id, size in enumerate(int_props):
                self.partitions[client_id].extend(idxs[start:start + size])
                start += size
            # If any samples are left over due to floor truncation, assign them
            # to clients with the largest initial share (or random if equal).  This
            # ensures that the union of partitions covers the full dataset.
            remaining = len(idxs) - start
            if remaining > 0:
                # Rank clients by proportion (descending); break ties randomly
                ranked_clients = np.argsort(-proportions)
                # Distribute leftover samples in round‑robin order among ranked clients
                for i in range(remaining):
                    cid = ranked_clients[i % len(ranked_clients)]
                    self.partitions[int(cid)].append(idxs[start + i])

        for cid in self.partitions:
            np.random.shuffle(self.partitions[cid])

        return self.partitions

    # ... rest of partitioning.py remains unchanged ...


    def get_client_data(self, client_id: int) -> Subset:
        """
        Retrieve dataset subset for a specific client.

        Args:
            client_id (int): Client identifier.
        """
        if self.partitions is None:
            raise ValueError("Please run distribute_data() before accessing client data.")
        indices = self.partitions[client_id]
        return Subset(self.train_dataset, indices)

    def visualize_distribution(self, save_path: str | None = None) -> None:
        """
        Create IEEE-style stacked bar chart of sample counts per client.

        Args:
            save_path (str | None): File path to save the visualization.
        """
        if self.partitions is None:
            raise ValueError("Run distribute_data() before visualization.")

        path_obj = Path(save_path) if save_path is not None else DEFAULT_RESULTS_DIR / "data_distribution_ieee.png"
        if not path_obj.is_absolute():
            path_obj = BASE_PATH / path_obj
        path_obj.parent.mkdir(parents=True, exist_ok=True)

        targets = np.array(self.train_dataset.targets)
        client_counts = np.zeros((len(self.partitions), self.num_classes), dtype=int)

        for cid, idxs in self.partitions.items():
            class_counts = np.bincount(targets[idxs], minlength=self.num_classes)
            client_counts[cid, :] = class_counts

        # IEEE single-column figure size (~3.5in wide)
        fig, ax = plt.subplots(figsize=(1.8, 1.2), dpi=300)
        bottom = np.zeros(len(self.partitions))
        colors = plt.get_cmap("tab20").colors

        for cls in range(self.num_classes):
            ax.bar(
                x=np.arange(len(self.partitions)),
                height=client_counts[:, cls],
                bottom=bottom,
                color=colors[cls % len(colors)],
                linewidth=0.1,
                edgecolor="white",
            )
            bottom += client_counts[:, cls]

        ax.set_xlabel("Client ID", fontsize=8)
        ax.set_ylabel("Samples", fontsize=8)
        ax.set_title(f"{self.dataset_name.upper()} Data Distribution Among Clients", fontsize=9)
        ax.tick_params(axis="both", which="major", labelsize=7)

        fig.savefig(path_obj, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"📊 Data distribution plot saved at: {path_obj}")


# -------------------------------
# Example Usage (for testing)
# -------------------------------
if __name__ == "__main__":
    distributor = DataDistributor(dataset_name="CIFAR10", data_dir=DEFAULT_DATA_DIR)
    distributor.distribute_data(num_clients=5, alpha=0.3, seed=42)
    distributor.visualize_distribution(DEFAULT_RESULTS_DIR / "cifar10_distribution_ieee.png")

    # Retrieve client dataset subset
    client_data = distributor.get_client_data(0)
    print(f"✅ Client 0 has {len(client_data)} samples.")


📊 Data distribution plot saved at: /content/drive/MyDrive/TrustWeightColab/results/cifar10_distribution_ieee.png
✅ Client 0 has 13080 samples.


In [58]:
# ===== strategy.py =====

# Trust-weighted asynchronous aggregation strategy implementing the PDF math
from __future__ import annotations

from dataclasses import dataclass
from typing import Dict, List, Tuple

import torch


@dataclass
class TrustWeightedConfig:
    eta: float = 1.0              # server learning rate η
    eps: float = 1e-8             # numerical stability ε
    freshness_alpha: float = 0.1  # α in s(τ) = exp(-α τ)
    beta1: float = 0.0            # Guard term coefficient on staleness
    beta2: float = 0.0            # Guard term coefficient on ||u||
    momentum_gamma: float = 0.9   # update factor for m_t
    theta: Tuple[float, float, float] = (0.0, 0.0, 0.0)  # quality weights


class TrustWeightedAsyncStrategy:
    """Implements the aggregation rule described in the DML solution PDF.

    Core formula:

        w_{t+1} = w_t + η * Σ_i Weight_i *
            [ Proj_m_t(u_i) + Guard_i * (u_i - Proj_m_t(u_i)) ]

    with:

        Proj_m_t(u_i) = <u_i, m_t> / (||m_t||^2 + eps) * m_t
        Guard_i = 1 / (1 + β1 * τ_i + β2 * ||u_i||)
        Weight_i ∝ s(τ_i) * exp(θᵀ [ΔL̃_i, ||u_i||, cos(u_i, m_t)]) * (n_i / Σ_j n_j)
    """

    def __init__(self, dim: int, cfg: TrustWeightedConfig | None = None) -> None:
        self.dim = int(dim)
        self.cfg = cfg or TrustWeightedConfig()
        self.m = torch.zeros(self.dim, dtype=torch.float32)  # m_t, server momentum
        self.step: int = 0

        self.theta = torch.tensor(self.cfg.theta, dtype=torch.float32)

    # ------------------------------------------------------------------ helpers

    def _proj_m(self, u: torch.Tensor) -> torch.Tensor:
        # Proj_m(u) = <u, m> / (||m||^2 + eps) * m
        num = torch.dot(u, self.m)
        denom = torch.dot(self.m, self.m) + self.cfg.eps
        coef = num / denom
        return coef * self.m

    def _guard(self, tau: torch.Tensor, norm_u: torch.Tensor) -> torch.Tensor:
        # Guard_i = 1 / (1 + β1 τ_i + β2 ||u_i||)
        return 1.0 / (1.0 + self.cfg.beta1 * tau + self.cfg.beta2 * norm_u)

    def _freshness(self, tau: torch.Tensor) -> torch.Tensor:
        # s(τ) = exp(-α τ)
        return torch.exp(-self.cfg.freshness_alpha * tau)

    # ---------------------------------------------------------------- aggregate

    def aggregate(
        self,
        w_t: torch.Tensor,
        updates: List[Dict[str, torch.Tensor]],
    ) -> Tuple[torch.Tensor, Dict[str, float]]:
        """Aggregate a buffer of updates.

        Args:
            w_t: Flattened current global model.
            updates: List of dicts, each containing:
                {
                    "u": update vector (1D tensor),
                    "tau": scalar tensor τ_i,
                    "num_samples": scalar tensor n_i,
                    "delta_loss": scalar tensor ΔL̃_i,
                }

        Returns:
            new_w: updated global model vector.
            metrics: small dict with aggregation statistics.
        """
        if not updates:
            return w_t, {"avg_tau": 0.0, "buffer_size": 0.0}

        self.step += 1
        device = w_t.device
        self.m = self.m.to(device)

        # Collect basic statistics
        taus = torch.stack([u["tau"].to(device) for u in updates])  # [B]
        ns = torch.stack([u["num_samples"].to(device) for u in updates])  # [B]
        delta_losses = torch.stack([u["delta_loss"].to(device) for u in updates])  # [B]
        total_n = ns.sum().clamp_min(1.0)

        # Precompute norms, projections, sideways components, cosines
        proj_list: List[torch.Tensor] = []
        side_list: List[torch.Tensor] = []
        norm_u_list: List[torch.Tensor] = []
        cos_list: List[torch.Tensor] = []

        for u_rec in updates:
            u = u_rec["u"].to(device)
            norm_u = torch.norm(u).clamp_min(self.cfg.eps)
            norm_u_list.append(norm_u)

            proj = self._proj_m(u)
            side = u - proj
            proj_list.append(proj)
            side_list.append(side)

            # cos(u, m) = <u, m> / (||u|| ||m|| + eps)
            norm_m = torch.norm(self.m)
            if norm_m.item() > 0.0:
                cos_val = torch.dot(u, self.m) / (norm_u * norm_m + self.cfg.eps)
            else:
                cos_val = torch.tensor(0.0, device=device)
            cos_list.append(cos_val)

        norm_u_tensor = torch.stack(norm_u_list)  # [B]
        cos_tensor = torch.stack(cos_list)  # [B]

        # Guard factors per update
        guards = self._guard(taus, norm_u_tensor)  # [B]

        # Freshness
        freshness = self._freshness(taus)  # [B]

        # Quality term: exp(θᵀ [ΔL̃_i, ||u_i||, cos(u_i, m_t)])
        feats = torch.stack(
            [delta_losses, norm_u_tensor, cos_tensor],
            dim=1,
        )  # [B, 3]
        quality_logits = feats @ self.theta.to(device)
        quality = torch.exp(quality_logits)

        # Data share term: n_i / Σ_j n_j
        data_share = ns / total_n  # [B]

        # Unnormalized weights, then normalization over buffer
        raw_weights = freshness * quality * data_share  # [B]
        sum_raw = raw_weights.sum()
        if sum_raw.item() <= 0.0:
            weights = torch.full_like(raw_weights, 1.0 / len(updates))
        else:
            weights = raw_weights / sum_raw

        # Combine projection and guarded sideways components
        agg_update = torch.zeros_like(w_t)
        for i in range(len(updates)):
            comp = proj_list[i] + guards[i] * side_list[i]
            agg_update = agg_update + weights[i] * comp

        # Final aggregation step:
        # w_{t+1} = w_t + η * Σ_i Weight_i * [Proj_m(u_i) + Guard_i (u_i - Proj_m(u_i))]
        new_w = w_t + self.cfg.eta * agg_update

        # Update momentum m_t as a running average of aggregated updates
        self.m = (1.0 - self.cfg.momentum_gamma) * self.m + self.cfg.momentum_gamma * agg_update

        metrics = {
            "avg_tau": float(taus.mean().item()),
            "avg_norm_u": float(norm_u_tensor.mean().item()),
            "avg_delta_loss": float(delta_losses.mean().item()),
            "buffer_size": float(len(updates)),
        }
        return new_w, metrics


In [59]:
# ===== server.py =====

# Asynchronous federated server implementing the Trust-Weighted projection rule
import csv
import time
import threading
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple, Optional, OrderedDict as ODType

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms



# ---------------------------------------------------------------------------

def _testloader(root: str, batch_size: int = 256) -> DataLoader:
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
        ]
    )
    ds = datasets.CIFAR10(root=root, train=False, download=True, transform=transform)
    # num_workers=0 to avoid multiprocessing / SemLock issues
    return DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=0)


def _flatten_state(state: "ODType[str, torch.Tensor]") -> torch.Tensor:
    """Flatten a state_dict into a 1D tensor.

    The default implementation simply concatenates parameters in whatever
    iteration order the dictionary exposes.  While this works for
    ``OrderedDict`` instances created from the same model template, it can
    become brittle when ``state`` is a plain Python ``dict`` because the
    insertion order may differ from the server's canonical template.  An
    inconsistent ordering corrupts the mapping between parameter slices and
    model layers, which in turn leads to meaningless update vectors and
    prevents the global model from learning.

    To avoid such subtle bugs, prefer ``_flatten_state_by_template`` when
    flattening a state that may not share the exact ordering with the
    server's template (see ``_flatten_state_by_template`` below).
    """
    return torch.cat([p.reshape(-1) for p in state.values()])


def _flatten_state_by_template(
    state: Dict[str, torch.Tensor], template: "ODType[str, torch.Tensor]"
) -> torch.Tensor:
    """Flatten ``state`` according to the key ordering of ``template``.

    Many downstream computations (e.g. computing parameter deltas) assume
    that all 1D parameter vectors follow the same ordering.  When
    ``state`` is a plain ``dict`` created from a model state_dict,
    Python preserves insertion order, but nothing guarantees that this
    ordering matches that of the server's ``template``.  This helper
    constructs a flattened tensor by explicitly iterating over the keys of
    ``template``, thereby aligning the parameter ordering regardless of
    how ``state`` was constructed.

    Args:
        state: Mapping from parameter names to tensors.  Can be a
            standard ``dict`` or ``OrderedDict``.
        template: The canonical parameter ordering to follow.

    Returns:
        A 1D tensor containing all parameters from ``state`` in the order
        specified by ``template``.
    """
    return torch.cat([state[k].reshape(-1) for k in template.keys()])


def _vector_to_state(
    vec: torch.Tensor, template: "ODType[str, torch.Tensor]"
) -> "ODType[str, torch.Tensor]":
    new_state: "ODType[str, torch.Tensor]" = type(template)()
    offset = 0
    for k, t in template.items():
        numel = t.numel()
        new_state[k] = vec[offset : offset + numel].view_as(t).clone()
        offset += numel
    assert offset == vec.numel()
    return new_state


@dataclass
class ClientUpdateState:
    client_id: int
    base_version: int
    new_params: "ODType[str, torch.Tensor]"
    num_samples: int
    train_time_s: float
    delta_loss: float
    loss_before: float
    loss_after: float
    train_acc: float
    test_loss: float
    test_acc: float
    arrival_ts: float


class AsyncServer:
    """Central server maintaining global model and asynchronous buffer."""

    def __init__(self, cfg: Optional[GlobalConfig] = None) -> None:
        if cfg is None:
            cfg = load_config()
        self.cfg = cfg
        self.device = get_device()

        # data / evaluation
        self.testloader = _testloader(cfg.data.data_dir, batch_size=256)

        # global model and version history
        model = build_resnet18(num_classes=cfg.data.num_classes)
        self._template_state: ODType[str, torch.Tensor] = ODType(
            (k, v.detach().cpu().clone()) for k, v in model.state_dict().items()
        )
        self._global_state: ODType[str, torch.Tensor] = ODType(
            (k, v.clone()) for k, v in self._template_state.items()
        )

        self._model_versions: List[ODType[str, torch.Tensor]] = [
            ODType((k, v.clone()) for k, v in self._global_state.items())
        ]
        self._version: int = 0

        # strategy encapsulating all math from the PDF
        dim = _flatten_state(self._global_state).numel()
        self.strategy = TrustWeightedAsyncStrategy(dim=dim)

        # async buffer
        self.buffer: List[ClientUpdateState] = []
        self.buffer_size: int = 5  # K
        self.buffer_timeout_s: float = 5.0  # Δt in seconds
        self._last_flush_ts: float = time.time()

        # logging / control
        self.io_root = Path(cfg.io.logs_dir)
        self.io_root.mkdir(parents=True, exist_ok=True)

        # Global training log CSV
        self.global_log_path = Path(cfg.io.global_log_csv)
        self.global_log_path.parent.mkdir(parents=True, exist_ok=True)
        self._init_global_log()

        # Client participation log CSV
        self.client_log_path = Path(cfg.io.client_participation_csv)
        self.client_log_path.parent.mkdir(parents=True, exist_ok=True)
        self._init_client_log()

        self.eval_interval_s: float = cfg.eval.interval_seconds
        self.target_accuracy: float = cfg.eval.target_accuracy
        self.max_rounds: int = cfg.train.max_rounds
        self.update_clip_norm: float = float(cfg.train.update_clip_norm)

        self._num_aggregations: int = 0  # total_agg counter
        self._stop: bool = False
        self._stop_reason: str = ""
        self._lock = threading.Lock()

        # Ensures only one aggregation is in-flight at a time.  Without
        # this lock, multiple threads could concurrently call
        # ``_aggregate`` on separate buffers, causing race conditions in
        # versioning and inconsistent staleness calculations.  All calls
        # to ``_aggregate`` must acquire this lock.
        self._agg_lock = threading.Lock()

    # ----------------------------------------------------------------- logging

    def _init_global_log(self) -> None:
        """Initialize the global training CSV if it does not exist.

        Columns (per aggregation):
            total_agg, avg_train_loss, avg_train_acc, test_loss, test_acc, time
        """
        if self.global_log_path.exists():
            return
        with self.global_log_path.open("w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(
                [
                    "total_agg",
                    "avg_train_loss",
                    "avg_train_acc",
                    "test_loss",
                    "test_acc",
                    "time",
                ]
            )

    def _append_global_log(
        self,
        total_agg: int,
        avg_train_loss: float,
        avg_train_acc: float,
        test_loss: float,
        test_acc: float,
    ) -> None:
        """Append a single aggregation row to the global CSV."""
        ts = time.time()
        with self.global_log_path.open("a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(
                [
                    int(total_agg),
                    float(avg_train_loss),
                    float(avg_train_acc),
                    float(test_loss),
                    float(test_acc),
                    ts,
                ]
            )

    def _init_client_log(self) -> None:
        """Initialize the client participation CSV.

        Columns:
            client_id, local_train_loss, local_train_acc,
            local_test_loss, local_test_acc, total_agg, staleness
        """
        if self.client_log_path.exists():
            return
        with self.client_log_path.open("w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(
                [
                    "client_id",
                    "local_train_loss",
                    "local_train_acc",
                    "local_test_loss",
                    "local_test_acc",
                    "total_agg",
                    "staleness",
                ]
            )

    def _append_client_participation_log(
        self,
        total_agg: int,
        updates: List[ClientUpdateState],
        staleness_list: List[float],
    ) -> None:
        """Append one row per client update participating in this aggregation.

        staleness_list[i] is the τ_i for updates[i].
        """
        with self.client_log_path.open("a", newline="") as f:
            writer = csv.writer(f)
            for u, tau_i in zip(updates, staleness_list):
                writer.writerow(
                    [
                        int(u.client_id),
                        float(u.loss_after),
                        float(u.train_acc),
                        float(u.test_loss),
                        float(u.test_acc),
                        int(total_agg),
                        float(tau_i),
                    ]
                )

    # ------------------------------------------------------------------- public

    def should_stop(self) -> bool:
        with self._lock:
            return self._stop

    def mark_stop(self, reason: str = "") -> None:
        with self._lock:
            if not self._stop:
                self._stop = True
                if reason:
                    self._stop_reason = reason
                print(f"[Server] Stopping: {self._stop_reason}")

    # --------------------------------------------------------------- model I/O

    def get_global_model(self) -> Tuple[int, Dict[str, torch.Tensor]]:
        """Return (version, state_dict) of the current global model."""
        with self._lock:
            version = self._version
            # Preserve the parameter ordering when sending to clients by
            # constructing an OrderedDict.  A plain dict may reorder keys
            # unexpectedly on some Python versions or implementations,
            # breaking downstream flatten/unflatten assumptions.  Clones
            # ensure the server's tensors remain unmodified.
            state = ODType((k, v.clone()) for k, v in self._global_state.items())
        return version, state

    # --------------------------------------------------------------- evaluation

    def _make_model_from_state(self, state: Dict[str, torch.Tensor]) -> torch.nn.Module:
        model = build_resnet18(num_classes=self.cfg.data.num_classes)
        model.load_state_dict(state)
        return model.to(self.device)

    def _evaluate_global(self) -> Tuple[float, float]:
        """Evaluate the current global model on the test set."""
        model = self._make_model_from_state(self._global_state)
        model.eval()
        criterion = torch.nn.CrossEntropyLoss()
        total_loss = 0.0
        total_correct = 0
        total_examples = 0
        with torch.no_grad():
            for xb, yb in self.testloader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                logits = model(xb)
                loss = criterion(logits, yb)
                total_loss += loss.item() * xb.size(0)
                preds = logits.argmax(dim=1)
                total_correct += (preds == yb).sum().item()
                total_examples += xb.size(0)
        if total_examples == 0:
            return 0.0, 0.0
        return total_loss / total_examples, total_correct / total_examples

    # --------------------------------------------------------------- aggregation

    def _flush_buffer_if_needed(self) -> None:
        now = time.time()
        should_flush = False
        if len(self.buffer) >= self.buffer_size:
            should_flush = True
        elif (now - self._last_flush_ts) >= self.buffer_timeout_s and self.buffer:
            should_flush = True

        if not should_flush:
            return

        # copy buffer locally under lock then release for heavy work
        with self._lock:
            buffer_copy = list(self.buffer)
            self.buffer.clear()
            self._last_flush_ts = now

        # Serialize aggregations to avoid version races
        with self._agg_lock:
            self._aggregate(buffer_copy)

    def _aggregate(self, updates: List[ClientUpdateState]) -> None:
        """Aggregate a batch of client updates and log to CSVs."""
        if not updates:
            return

        # Snapshot of current global parameters and version history
        with self._lock:
            # Always flatten the global state according to the template order
            global_vec = _flatten_state_by_template(self._global_state, self._template_state)
            version_now = self._version
            model_versions = list(self._model_versions)

        # Construct per-update vectors and metadata for the strategy,
        # and collect staleness τ_i for logging.
        update_vectors: List[Dict[str, torch.Tensor]] = []
        staleness_list: List[float] = []
        valid_updates: List[ClientUpdateState] = []

        for u in updates:
            base_state = model_versions[u.base_version]
            # Flatten base_state and new_params using the canonical template order
            base_vec = _flatten_state_by_template(base_state, self._template_state)
            new_vec = _flatten_state_by_template(u.new_params, self._template_state)
            ui = new_vec - base_vec

            # Skip obviously bad updates (NaN/Inf) to avoid corrupting the global model
            if not torch.isfinite(ui).all():
                print(f"[Server] Dropping client {u.client_id} update due to NaN/Inf values")
                continue
            if self.update_clip_norm > 0:
                norm = torch.norm(ui)
                if torch.isfinite(norm) and norm.item() > self.update_clip_norm:
                    ui = ui * (self.update_clip_norm / (norm + 1e-12))

            # τ_i = current-server-version - base_version (same τ_i used in strategy)
            tau_i = float(max(0, version_now - u.base_version))
            staleness_list.append(tau_i)

            delta_loss = float(u.delta_loss)  # ΔL̃_i
            update_vectors.append(
                {
                    "u": ui,
                    "tau": torch.tensor(tau_i, dtype=torch.float32),
                    "num_samples": torch.tensor(float(u.num_samples), dtype=torch.float32),
                    "delta_loss": torch.tensor(delta_loss, dtype=torch.float32),
                }
            )
            valid_updates.append(u)

        if not update_vectors:
            print("[Server] Buffer flush skipped: no valid updates after filtering.")
            return

        # Run the trust-weighted aggregation strategy (unchanged algorithm)
        new_global_vec, agg_metrics = self.strategy.aggregate(global_vec, update_vectors)

        # Map back into parameter state_dict form
        new_state = _vector_to_state(new_global_vec, self._template_state)

        # Compute average local train metrics for this aggregation
        avg_train_loss = sum(u.loss_after for u in valid_updates) / len(valid_updates)
        avg_train_acc = sum(u.train_acc for u in valid_updates) / len(valid_updates)

        # Commit the new global model and update aggregation counter
        with self._lock:
            self._global_state = ODType((k, v.clone()) for k, v in new_state.items())
            self._model_versions.append(
                ODType((k, v.clone()) for k, v in self._global_state.items())
            )
            self._version = len(self._model_versions) - 1
            self._num_aggregations += 1
            total_agg = self._num_aggregations

        # Evaluate updated global model on test data
        test_loss, test_acc = self._evaluate_global()

        # Log global metrics and per-client participation (now with staleness)
        self._append_global_log(
            total_agg=total_agg,
            avg_train_loss=avg_train_loss,
            avg_train_acc=avg_train_acc,
            test_loss=test_loss,
            test_acc=test_acc,
        )
        self._append_client_participation_log(
            total_agg=total_agg,
            updates=valid_updates,
            staleness_list=staleness_list,
        )

        print(
            f"[Server] Aggregated {len(valid_updates)} updates -> agg={total_agg} "
            f"(avg_tau={agg_metrics.get('avg_tau', 0.0):.3f}, "
            f"test_loss={test_loss:.4f}, test_acc={test_acc:.4f})"
        )

        # Stopping conditions based on global performance and max rounds
        if test_acc >= self.target_accuracy:
            self.mark_stop(f"target accuracy {test_acc:.4f} reached")
        if total_agg >= self.max_rounds:
            self.mark_stop("max aggregation rounds reached")

    # ------------------------------------------------------------ client entry

    def submit_update(
        self,
        client_id: int,
        base_version: int,
        new_params: Dict[str, torch.Tensor],
        num_samples: int,
        train_time_s: float,
        delta_loss: float,
        loss_before: float,
        loss_after: float,
        train_acc: float,
        test_loss: float,
        test_acc: float,
    ) -> None:
        """Entry point called by clients after local training.

        This method only enqueues the update and triggers buffer flushing.
        All CSV logging tied to a particular aggregation happens inside
        `_aggregate` so that `total_agg` and `staleness` are consistent.
        """
        cu = ClientUpdateState(
            client_id=client_id,
            base_version=base_version,
            new_params=new_params,
            num_samples=num_samples,
            train_time_s=float(train_time_s),
            delta_loss=float(delta_loss),
            loss_before=float(loss_before),
            loss_after=float(loss_after),
            train_acc=float(train_acc),
            test_loss=float(test_loss),
            test_acc=float(test_acc),
            arrival_ts=time.time(),
        )
        with self._lock:
            self.buffer.append(cu)
        self._flush_buffer_if_needed()

    # --------------------------------------------------------------- lifecycle

    def wait(self) -> None:
        """Block until training is finished (stopping condition reached)."""
        try:
            while not self.should_stop():
                time.sleep(0.2)
        finally:
            self.mark_stop(self._stop_reason or "training finished")


In [60]:
# ===== client.py =====

# Lightning-free async federated client for CIFAR-10
import time
import random
from typing import Sequence, Tuple

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torch.nn.utils import clip_grad_norm_

import pytorch_lightning as pl  # imported but not required; kept for compatibility
from pytorch_lightning.callbacks import ModelCheckpoint  # unused, kept for compatibility



def _build_transform() -> transforms.Compose:
    # CIFAR-10 standard augmentation to reduce overfitting
    return transforms.Compose(
        [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
        ]
    )


def _make_dataloader(
    data_dir: str,
    indices: Sequence[int],
    batch_size: int,
) -> DataLoader:
    dataset = datasets.CIFAR10(
        root=data_dir,
        train=True,
        download=True,
        transform=_build_transform(),
    )
    subset = Subset(dataset, indices)

    # Safety: in case partitioning ever returns an empty list for a client
    if len(subset) == 0:
        return DataLoader(subset, batch_size=1, shuffle=False, num_workers=0)

    # num_workers=0 to avoid multiprocessing issues on macOS / Python 3.13
    return DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=0)


class AsyncClient:
    """Asynchronous client performing local training on its partition.

    The client fetches the latest global model from the server, trains for a few
    local epochs, and submits the updated parameters plus basic telemetry.
    """

    def __init__(
        self,
        cid: int,
        indices: Sequence[int],
        cfg: GlobalConfig,
    ) -> None:
        self.cid = cid
        self.cfg = cfg
        self.device = get_device()
        self.loader = _make_dataloader(
            cfg.data.data_dir,
            indices,
            cfg.clients.batch_size,
        )
        self.num_classes = cfg.data.num_classes
        self.local_epochs = cfg.clients.local_epochs
        self.lr = cfg.clients.lr
        self.weight_decay = cfg.clients.weight_decay
        self.grad_clip = cfg.clients.grad_clip

        # --- straggler behaviour ---
        num_clients = cfg.clients.total
        slow_fraction = cfg.clients.struggle_percent / 100.0
        num_slow = int(round(num_clients * slow_fraction))
        self.is_slow = cid < num_slow  # deterministic but good enough

        self.delay_slow_range = tuple(cfg.clients.delay_slow_range)
        self.delay_fast_range = tuple(cfg.clients.delay_fast_range)
        self.jitter_per_round = float(cfg.clients.jitter_per_round)
        self.client_delay = float(cfg.server_runtime.client_delay)

    # ------------------------------------------------------------------ utils

    def _sample_delay(self) -> float:
        if self.is_slow:
            base = random.uniform(*self.delay_slow_range)
        else:
            base = random.uniform(*self.delay_fast_range)
        jitter = random.uniform(-self.jitter_per_round, self.jitter_per_round)
        return max(0.0, base + jitter + self.client_delay)

    def _build_model(self) -> nn.Module:
        model = build_resnet18(num_classes=self.num_classes)
        return model.to(self.device)

    # ---------------------------------------------------------------- training

    def _evaluate_on_loader(self, model: nn.Module) -> Tuple[float, float]:
        """Return (loss, accuracy) on the client's local data."""
        model.eval()
        criterion = nn.CrossEntropyLoss()
        total_loss = 0.0
        total_correct = 0
        total_examples = 0
        with torch.no_grad():
            for xb, yb in self.loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                logits = model(xb)
                loss = criterion(logits, yb)
                total_loss += loss.item() * xb.size(0)
                preds = logits.argmax(dim=1)
                total_correct += (preds == yb).sum().item()
                total_examples += xb.size(0)
        if total_examples == 0:
            return 0.0, 0.0
        return total_loss / total_examples, total_correct / total_examples

    def _train_local(self, model: nn.Module) -> Tuple[float, float]:
        """Train for `local_epochs` and return (loss_after, acc_after)."""
        model.train()
        criterion = nn.CrossEntropyLoss()
        optim = torch.optim.SGD(
            model.parameters(),
            lr=self.lr,
            momentum=0.9,
            weight_decay=self.weight_decay,
        )

        for _ in range(self.local_epochs):
            for xb, yb in self.loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                optim.zero_grad(set_to_none=True)
                logits = model(xb)
                loss = criterion(logits, yb)
                loss.backward()
                if self.grad_clip > 0:
                    clip_grad_norm_(model.parameters(), self.grad_clip)
                optim.step()

        # reuse evaluation code for final metrics
        return self._evaluate_on_loader(model)

    # ------------------------------------------------------------- main loop

    def run_once(self, server) -> bool:
        """Perform a single async round with the server.

        Returns False when the server indicates global stopping, True otherwise.
        """
        # Simulated network / computation delay heterogeneity
        delay = self._sample_delay()
        if delay > 0:
            time.sleep(delay)

        if server.should_stop():
            return False

        # Get the latest global model snapshot
        version, global_state = server.get_global_model()
        model = self._build_model()
        model.load_state_dict(global_state)

        # Evaluate before local training to compute loss drop ΔL̃_i
        loss_before, _ = self._evaluate_on_loader(model)

        start_time = time.time()
        loss_after, train_acc = self._train_local(model)
        train_time_s = time.time() - start_time

        # Local "test" is just another pass over the client's data
        test_loss, test_acc = self._evaluate_on_loader(model)

        # Move params to CPU tensors so they are cheap to share with server
        # Build an OrderedDict for `new_params` using the model's own
        # parameter ordering.  Although Python's plain dicts preserve
        # insertion order, explicitly constructing an `OrderedDict` makes
        # the intent clear and avoids any surprises if the language
        # specification changes.  The server relies on matching key
        # ordering to correctly flatten parameter tensors.
        from collections import OrderedDict
        new_params = OrderedDict()
        for k, v in model.state_dict().items():
            new_params[k] = v.detach().cpu().clone()
        num_examples = len(self.loader.dataset)

        delta_loss = loss_before - loss_after  # ΔL̃_i

        # All local metrics are computed here and passed to the server
        server.submit_update(
            client_id=self.cid,
            base_version=version,
            new_params=new_params,
            num_samples=num_examples,
            train_time_s=train_time_s,
            delta_loss=delta_loss,
            loss_before=loss_before,
            loss_after=loss_after,
            train_acc=train_acc,
            test_loss=test_loss,
            test_acc=test_acc,
        )
        return not server.should_stop()

In [ ]:
# ===== experiment.py =====

# Experiment runner for TrustWeight sweeps (partition alpha and straggler percent)
import logging
from copy import deepcopy
from dataclasses import replace
from pathlib import Path
import math
import csv
from typing import Iterable, List
import threading


def _safe_name(val) -> str:
    """Convert numeric value to a filesystem-friendly token."""
    s = str(val)
    return s.replace(".", "p")


def _override_io(cfg: GlobalConfig, exp_dir: Path) -> GlobalConfig:
    """Return a copy of cfg with I/O paths pointing inside exp_dir."""
    exp_dir = exp_dir.resolve()
    io = cfg.io
    new_io = replace(
        io,
        logs_dir=str(exp_dir),
        checkpoints_dir=str(exp_dir / "checkpoints"),
        results_dir=str(exp_dir / "results"),
        global_log_csv=str(exp_dir / "TrustWeight.csv"),
        client_participation_csv=str(exp_dir / "TrustWeightClientParticipation.csv"),
        final_model_path=str(exp_dir / "TrustWeightModel.pt"),
    )
    return replace(cfg, io=new_io)


def run_with_cfg(cfg: GlobalConfig) -> None:
    """Run one TrustWeight training session with an already-mutated cfg."""
    # --------------------- seed ---------------------
    logging.basicConfig(level=logging.INFO)

    # --------------------- dataset & partition --------------------
    distributor = DataDistributor(
        dataset_name=cfg.data.dataset,
        data_dir=cfg.data.data_dir,
    )
    distributor.distribute_data(
        num_clients=cfg.clients.total,
        alpha=cfg.partition_alpha,
        seed=cfg.seed,
    )
    partitions = [distributor.partitions[cid] for cid in range(cfg.clients.total)]

    # --------------------- create server -------------------------
    server = AsyncServer(cfg=cfg)

    # --------------------- create clients ------------------------
    clients: List[AsyncClient] = []
    for cid in range(cfg.clients.total):
        indices = partitions[cid] if cid < len(partitions) else []
        clients.append(AsyncClient(cid=cid, indices=indices, cfg=cfg))

    # ------------------- concurrency control ---------------------
    sem = threading.Semaphore(cfg.clients.concurrent)

    def client_loop(cl: AsyncClient) -> None:
        while not server.should_stop():
            with sem:
                cont = cl.run_once(server)
            if not cont or server.should_stop():
                break

    # --------------------- start client threads ------------------
    threads: List[threading.Thread] = []
    for cl in clients:
        t = threading.Thread(target=client_loop, args=(cl,), daemon=True)
        t.start()
        threads.append(t)

    # --------------------- wait for completion -------------------
    server.wait()
    for t in threads:
        t.join(timeout=1.0)


def alpha_sweep(
    base_cfg: GlobalConfig,
    alphas: Iterable[float],
    out_root: Path,
) -> None:
    """Run multiple trainings varying the Dirichlet alpha."""
    for alpha in alphas:
        exp_dir = out_root / f"alpha_{_safe_name(alpha)}"
        cfg = deepcopy(base_cfg)
        cfg.partition_alpha = float(alpha)
        cfg = _override_io(cfg, exp_dir)
        print(f"[exp] alpha={alpha} -> logs at {exp_dir}")
        run_with_cfg(cfg)


def straggler_sweep(
    base_cfg: GlobalConfig,
    percents: Iterable[int],
    out_root: Path,
) -> None:
    """Run multiple trainings varying the percent of slow clients."""
    for pct in percents:
        exp_dir = out_root / f"straggle_{pct}pct"
        cfg = deepcopy(base_cfg)
        cfg.clients.struggle_percent = int(pct)
        cfg = _override_io(cfg, exp_dir)
        print(f"[exp] struggle_percent={pct}% -> logs at {exp_dir}")
        run_with_cfg(cfg)


def client_sweep(
    base_cfg: GlobalConfig,
    client_counts: Iterable[int],
    out_root: Path,
) -> None:
    """Run multiple trainings varying the total number of clients."""
    for n_clients in client_counts:
        exp_dir = out_root / f"clients_{_safe_name(n_clients)}"
        cfg = deepcopy(base_cfg)

        # Set total number of clients
        cfg.clients.total = int(n_clients)

        # Optionally, you could also cap concurrent clients here if you want:
        # cfg.clients.concurrent = min(cfg.clients.concurrent, cfg.clients.total)

        cfg = _override_io(cfg, exp_dir)
        print(f"[exp] clients={n_clients} -> logs at {exp_dir}")
        run_with_cfg(cfg)



def sanity_check(exp_root: Path) -> None:
    """Lightweight sanity check over all experiment folders.

    Confirms required CSVs exist and contain finite test_acc values.
    Prints a short summary of best/last test_acc for quick inspection.
    """
    exp_root = exp_root.resolve()
    if not exp_root.exists():
        print(f"[sanity] No experiment folder found at {exp_root}")
        return

    exp_dirs = [p for p in exp_root.iterdir() if p.is_dir()]
    if not exp_dirs:
        print(f"[sanity] No sub-experiments found in {exp_root}")
        return

    for exp in sorted(exp_dirs):
        global_csv = exp / "TrustWeight.csv"
        client_csv = exp / "TrustWeightClientParticipation.csv"

        missing = [p.name for p in [global_csv, client_csv] if not p.exists()]
        if missing:
            print(f"[sanity] {exp.name}: missing {missing}")
            continue

        try:
            rows = []
            with global_csv.open() as f:
                reader = csv.DictReader(f)
                for row in reader:
                    rows.append(row)
        except Exception as e:
            print(f"[sanity] {exp.name}: failed to read CSV ({e})")
            continue

        if not rows:
            print(f"[sanity] {exp.name}: global CSV is empty")
            continue

        def _safe_float(x):
            try:
                v = float(x)
                return v if math.isfinite(v) else None
            except Exception:
                return None

        test_accs = [_safe_float(r.get("test_acc")) for r in rows]
        test_accs = [v for v in test_accs if v is not None]

        if not test_accs:
            print(f"[sanity] {exp.name}: no finite test_acc values")
            continue

        best = max(test_accs)
        last = test_accs[-1]
        print(f"[sanity] {exp.name}: best_test_acc={best:.4f}, last_test_acc={last:.4f}, n_rows={len(rows)}")


if __name__ == "__main__":
    base = load_config()

    client_root = Path(base.io.logs_dir) / "TrustWeightClientCountExp"
    client_root.mkdir(parents=True, exist_ok=True)

    # Experiment 3: client count sweep
    client_sweep(base, client_counts=[20, 40, 80], out_root=client_root)




[exp] clients=20 -> logs at /content/drive/MyDrive/TrustWeightColab/logs/TrustWeightClientCountExp/clients_20


100%|██████████| 170M/170M [00:01<00:00, 103MB/s]


[Server] Aggregated 1 updates -> agg=1 (avg_tau=0.000, test_loss=2.4572, test_acc=0.1000)
[Server] Aggregated 5 updates -> agg=2 (avg_tau=1.000, test_loss=2.3844, test_acc=0.1004)
[Server] Aggregated 5 updates -> agg=3 (avg_tau=2.000, test_loss=2.3421, test_acc=0.1014)
[Server] Aggregated 5 updates -> agg=4 (avg_tau=2.400, test_loss=2.3167, test_acc=0.1145)
[Server] Aggregated 3 updates -> agg=5 (avg_tau=3.000, test_loss=2.2997, test_acc=0.1325)
[Server] Aggregated 1 updates -> agg=6 (avg_tau=3.000, test_loss=2.2899, test_acc=0.1383)
[Server] Aggregated 3 updates -> agg=7 (avg_tau=4.000, test_loss=2.2819, test_acc=0.1422)
[Server] Aggregated 3 updates -> agg=8 (avg_tau=5.000, test_loss=2.2748, test_acc=0.1441)
[Server] Aggregated 1 updates -> agg=9 (avg_tau=5.000, test_loss=2.2693, test_acc=0.1435)
[Server] Aggregated 1 updates -> agg=10 (avg_tau=5.000, test_loss=2.2650, test_acc=0.1438)
[Server] Aggregated 4 updates -> agg=11 (avg_tau=6.000, test_loss=2.2619, test_acc=0.1422)
[Server]